# Препроцессинг диалоговых данных 

## В этом ноутбуке:

* Все вопросы-ответы (пара) заносятся в один датафрейм **raw_data** с указанием комитета (по возможности) и созыва Госдумы
* В датафрейм **processed_data** к raw_data добавлены косинусное расстояние между эмбеддингами вопроса и ответа, а также, как для вопроса, так и для ответа, доля того, что текст эмоционально-нейтральный, эмоционально-негативный, эмоционально-положительный. (а также информация о том, насколько модель уверена в своем результате, и какова доля "пустых" разговорных фраз)
* Датафрейм **processed_data_nameless** возвращает то же, что и *processed_data*, но для получения численных значений использует тексты без имен и отчеств (которые присутствуют практически в каждом диалоге в Думе).

In [13]:
!pip install torch >> None
!pip install transformers >> None
!pip install transformers sentencepiece >> None

!pip install Dostoevsky >> None
!python -m dostoevsky download fasttext-social-network-model >> None

!pip install natasha >> None

In [14]:
# библиотеки для работы с моделями платформы HuggingFace
import torch
from transformers import AutoTokenizer, AutoModel

# библиотеки для работы с эмбеддингами и текстами
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

# библиотеки для получения векторных представлений 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# библиотека Dostoevsky для задачи sentiment analysis
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

# Библиотека Natasha для получения именных представлений
# (named entity recognition)
from natasha import NamesExtractor

# библиотеки для загрузки данных из GitHub
import os
import re
import glob
import requests

# библиотеки для анализа данных
import numpy as np
import pandas as pd

# вспомогательные библиотеки
import warnings
# FutureWarnings предупреждения
warnings.filterwarnings("ignore", category=FutureWarning)

In [15]:
# Удалим данные, если они уже загружены локально

!rm -rf Scrapping_stenograms

# Загрузим данные из подготовленного репозитория Guthub

! git clone https://github.com/AleksandraVasilieva/Scrapping_stenograms/ 

Cloning into 'Scrapping_stenograms'...
remote: Enumerating objects: 348, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 348 (delta 20), reused 94 (delta 13), pack-reused 220
Receiving objects: 100% (348/348), 2.99 MiB | 15.60 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [16]:
nltk.download('punkt')
nltk.download("stopwords")

# загрузка стоп слов, которые удаляются из текстов
stops = stopwords.words('russian')
# стеммер
stemmer = SnowballStemmer('russian')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
def custom_tokenizer(x):
    # приведем все символы строки в нижний регистр и токенизируем строку: 
    #разобьем ее на отдельные слова
    step1 = word_tokenize(x)
    # уберем цифры и пунктуацию
    step2 = [i.lower() for i in step1 if i.isalpha()]
    # уберем стоп-слова
    step3 = [word for word in step2 if not word in stops]
    # проведем стемминг: убрем аффиксы
    step4 = [stemmer.stem(w) for w in step3]
    return step4

In [54]:
# Токенизатор и модель для Sentiment Analysis
dostoevsky_tokenizer = RegexTokenizer()
dostoevsky_sentiment_model = FastTextSocialNetworkModel(tokenizer=dostoevsky_tokenizer)
# results = dostoevsky_sentiment_model.predict([text])

# Токенизатор и модель для получения эмбеддингов
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
rubert_embeddings_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

# Модель Наташа для удаления имен
morph_vocab = MorphVocab()
natasha_names_extractor = NamesExtractor(morph_vocab)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
type(dostoevsky_sentiment_model.predict(['hello cat dog well no'])[0])

dict

In [20]:
# Функция для получения косинусного расстояния между эмбеддингами
def calculate_cosine_distance(embedding1, embedding2):
    distance = cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))
    similarity = 1 - distance
    return similarity[0][0]

# Функция для получения эмбеддинга текста
def get_embeddings(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [74]:
# Функция для получения косинусного расстояния между двумя необработанными текстами
def get_cosine_similarity(question, answer):
  '''
  Функция принимает на вход вопрос и ответ,
  токенизирует, получает векторное представление
  (эмбеддинги) и возвращает косинусное расстояние.

  '''
  # Получение эмбеддингов
  embedding_question = get_embeddings(question, rubert_embeddings_model, rubert_tokenizer)
  embedding_answer = get_embeddings(answer, rubert_embeddings_model, rubert_tokenizer)

  # Получение косинусного расстояния

  cosine_distance = calculate_cosine_distance(embedding_question, embedding_answer)
  
  return cosine_distance

In [73]:
# Функция для удаления имен в строке
def nameless_text(text):
  '''
  Функция вернет текст без имен и отчеств
  '''

  # Найдем имена
  matches = list(natasha_names_extractor(text))

  # Извлечем имена из текста
  for match in matches:
      start = match.start
      end = match.stop
      text = text[:start] + text[end:]

  return text

In [69]:
# создадим датафрейм для сырого текста
raw_data = pd.DataFrame(columns=['Duma Convocation', 'Commitee name', 'Question', 'Answer'])

# создадим датафрейм с добавочной информацией

processed_data = pd.DataFrame(columns=['Duma Convocation', 'Commitee name', 'Question', 'Answer', \
        'Cosine Similarity', 'q_neutral', 'q_negative', 'q_positive', 'q_unsure', 'q_colloquial',
        'a_neutral', 'a_negative', 'a_positive', 'a_unsure', 'a_colloquial'])

root_path = '/content/Scrapping_stenograms/dialogs data/'

# Пройдем по каждой папке
folders = ['duma 6/', 'duma 7/', 'duma 8/'] 
for folder in folders:
    session_name = folder[:-1]
    folder_link = root_path + folder

    # Узнаем, какие файлы в ней лежат
    files = glob.glob(folder_link + '/*')

    # Пройдем по каждому файл из текущей папки
    for file in files:
        commitee_name = file[len(file) - file[::-1].find('_'):][:-4]

        # Прочитаем файл
        with open(file, 'r', encoding='utf-8') as f:
          lines = f.readlines()

        # Пропустим файл, если он пустой
        if len(lines) == 0:
            continue

        # Будем считывать каждые 2 строки
        for i in range(0, len(lines), 2):
          question = lines[i].strip()
          answer = lines[i+1].strip()

          # вычислим косинусное расстояние между эмбеддингами
          cos_similarity = get_cosine_similarity(question, answer)

          # вычислим для вопроса и ответа их sentiment
          sentiment_dict_question = (dostoevsky_sentiment_model.predict([question]))[0]
          sentiment_dict_answer = (dostoevsky_sentiment_model.predict([answer]))[0]
          q_neutral = sentiment_dict_question['neutral']
          q_negative = sentiment_dict_question['negative']
          q_positive = sentiment_dict_question['positive']
          q_unsure = sentiment_dict_question['skip']
          q_colloquial = sentiment_dict_question['speech']
          a_neutral = sentiment_dict_answer['neutral']
          a_negative = sentiment_dict_answer['negative']
          a_positive = sentiment_dict_answer['positive']
          a_unsure = sentiment_dict_answer['skip']
          a_colloquial = sentiment_dict_answer['speech']


          # Добавим данные в датафрейм processed_data
          processed_data = processed_data.append({'Duma Convocation':session_name, \
          'Commitee name':commitee_name, 'Question': question, \
          'Answer': answer,'Cosine Similarity' : cos_similarity,  \
          'q_neutral': q_neutral, 'q_negative': q_negative,  \
          'q_positive':q_positive, \
          'q_unsure':q_unsure, 'q_colloquial':q_colloquial,\
          'a_neutral':a_neutral, 'a_negative':a_negative, \
          'a_positive':a_positive, 'a_unsure':a_unsure, 
          'a_colloquial': a_colloquial},\
          ignore_index=True)
            
          # Добавим данные в датафрейм raw_data
          raw_data = raw_data.append({'Duma Convocation':session_name, \
                          'Commitee name':commitee_name, \
                          'Question': question, 'Answer': answer}, ignore_index=True)
        

In [25]:
raw_data

,Duma Convocation,Commitee name,Question,Answer
0,duma 6,zero,У меня вопрос к Александру Александровичу Реме...,"Андрей Викторович, спасибо за вопрос. Да, дейс..."
1,duma 6,zero,У меня вопрос к Михаилу Васильевичу.Михаил Вас...,"Уважаемый Владимир Николаевич, вопрос о национ..."
2,duma 6,zero,"Вопрос представителю комитета, два вопроса. Пе...","Уважаемый Валентин Степанович, регионы, конечн..."
3,duma 6,zero,"Михаил Васильевич, в дополнение к уже сказанно...","Уважаемый Дмитрий Иванович, я думаю, что ваше ..."
4,duma 6,zero,"Спасибо.Уважаемый Александр Александрович, коп...",Что касается анализа комитета. Исходя из анали...
...,...,...,...,...
1747,duma 8,trudu,"Я хотела бы, чтобы вы уточнили. Объём получате...",Спасибо за вопросы.В государственной управляющ...
1748,duma 8,trudu,"Вячеслав Викторович, можно, да? Спасибо. Три м...","Уважаемый Вячеслав Викторович, уважаемые колле..."
1749,duma 8,trudu,"Уважаемый Вячеслав Викторович, на самом деле б...","Уважаемый Вячеслав Викторович, уважаемый Иван ..."
1750,duma 8,trudu,"Уважаемый Иван Иванович, уважаемые коллеги! Ко...","Николай Иванович, спасибо за вопрос. У несовер..."


In [24]:
processed_data[:10]

,Duma Convocation,Commitee name,Question,Answer,Cosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial
0,duma 6,zero,У меня вопрос к Александру Александровичу Реме...,"Андрей Викторович, спасибо за вопрос. Да, дейс...",0.241192,0.577505,0.201823,0.069552,0.112805,0.009136,0.665421,0.125933,0.098089,0.055015,0.021625
1,duma 6,zero,У меня вопрос к Михаилу Васильевичу.Михаил Вас...,"Уважаемый Владимир Николаевич, вопрос о национ...",0.159109,0.531219,0.445540,0.040856,0.098089,0.024433,0.839744,0.182436,0.034110,0.042098,0.025189
2,duma 6,zero,"Вопрос представителю комитета, два вопроса. Пе...","Уважаемый Валентин Степанович, регионы, конечн...",0.271291,0.812877,0.212079,0.013233,0.098089,0.008072,0.685959,0.196836,0.080367,0.065615,0.013647
3,duma 6,zero,"Михаил Васильевич, в дополнение к уже сказанно...","Уважаемый Дмитрий Иванович, я думаю, что ваше ...",0.179641,0.692652,0.250923,0.043376,0.085109,0.011342,0.749097,0.281416,0.051855,0.085109,0.011342
4,duma 6,zero,"Спасибо.Уважаемый Александр Александрович, коп...",Что касается анализа комитета. Исходя из анали...,0.317797,0.743178,0.144159,0.035155,0.055015,0.025967,0.839744,0.222710,0.032111,0.053413,0.008857
5,duma 6,zero,"Вопрос Белякову.Уважаемый Сергей Юрьевич, в по...","Логики никакой не было, это действительно была...",0.292106,0.822199,0.256842,0.018557,0.069552,0.012831,0.461027,0.377551,0.156115,0.051855,0.012442
6,duma 6,zero,"Это не российское ноу-хау, весь мир идёт по пу...",...которая завозится на территорию магаданской...,0.197428,0.615098,0.250923,0.067557,0.092698,0.026769,0.644235,0.245095,0.140346,0.048868,0.010997
7,duma 6,zero,"Уважаемый Иван Иванович, уважаемые депутаты! О...","Спасибо.Уважаемый Олег Евгеньевич, я, в общем-...",0.229306,0.766304,0.217348,0.071601,0.051855,0.009136,0.554480,0.125933,0.100889,0.053413,0.030225
8,duma 6,zero,"Николай Васильевич, мы много размышляли, на са...","Сергей Владимирович, это вообще не власть, это...",0.396177,0.538993,0.164526,0.071601,0.160276,0.014074,0.839744,0.348655,0.036230,0.022987,0.022987
9,duma 6,zero,"Спасибо, Иван Иванович.У меня вопрос к Пантеле...","Виктор Евграфович, спасибо за вопрос. Пока ещё...",0.279978,0.679189,0.122533,0.109716,0.055015,0.013647,0.585111,0.212079,0.026769,0.106701,0.009423


In [77]:
# создадим датафрейм с добавочной информацией, где из текстов убираются имена

processed_data_nameless = pd.DataFrame(columns=['Duma Convocation', 'Commitee name', 'Question', 'Answer', \
        'Cosine Similarity', 'q_neutral', 'q_negative', 'q_positive', 'q_unsure', 'q_colloquial',
        'a_neutral', 'a_negative', 'a_positive', 'a_unsure', 'a_colloquial'])

root_path = '/content/Scrapping_stenograms/dialogs data/'

# Пройдем по каждой папке
folders = ['duma 6/', 'duma 7/', 'duma 8/'] 
for folder in folders:
    session_name = folder[:-1]
    folder_link = root_path + folder

    # Узнаем, какие файлы в ней лежат
    files = glob.glob(folder_link + '/*')

    # Пройдем по каждому файл из текущей папки
    for file in files:
        commitee_name = file[len(file) - file[::-1].find('_'):][:-4]

        # Прочитаем файл
        with open(file, 'r', encoding='utf-8') as f:
          lines = f.readlines()

        # Пропустим файл, если он пустой
        if len(lines) == 0:
            continue

        # Будем считывать каждые 2 строки
        for i in range(0, len(lines), 2):
          question = lines[i].strip()
          answer = lines[i+1].strip()

          question_nameless = nameless_text(question)
          answer_nameless = nameless_text(answer)

          # вычислим косинусное расстояние между эмбеддингами
          cos_similarity = get_cosine_similarity(question_nameless, answer_nameless)

          # вычислим для вопроса и ответа их sentiment
          sentiment_dict_question = (dostoevsky_sentiment_model.predict([question_nameless]))[0]
          sentiment_dict_answer = (dostoevsky_sentiment_model.predict([answer_nameless]))[0]
          q_neutral = sentiment_dict_question['neutral']
          q_negative = sentiment_dict_question['negative']
          q_positive = sentiment_dict_question['positive']
          q_unsure = sentiment_dict_question['skip']
          q_colloquial = sentiment_dict_question['speech']
          a_neutral = sentiment_dict_answer['neutral']
          a_negative = sentiment_dict_answer['negative']
          a_positive = sentiment_dict_answer['positive']
          a_unsure = sentiment_dict_answer['skip']
          a_colloquial = sentiment_dict_answer['speech']


          # Добавим данные в датафрейм processed_data_nameless
          processed_data_nameless = processed_data_nameless.append({'Duma Convocation':session_name, \
          'Commitee name':commitee_name, 'Question': question, \
          'Answer': answer,'Cosine Similarity' : cos_similarity,  \
          'q_neutral': q_neutral, 'q_negative': q_negative,  \
          'q_positive':q_positive, \
          'q_unsure':q_unsure, 'q_colloquial':q_colloquial,\
          'a_neutral':a_neutral, 'a_negative':a_negative, \
          'a_positive':a_positive, 'a_unsure':a_unsure, 
          'a_colloquial': a_colloquial},\
          ignore_index=True)


In [78]:
processed_data_nameless[:10]

,Duma Convocation,Commitee name,Question,Answer,Cosine Similarity,q_neutral,q_negative,q_positive,q_unsure,q_colloquial,a_neutral,a_negative,a_positive,a_unsure,a_colloquial
0,duma 6,zero,У меня вопрос к Александру Александровичу Реме...,"Андрей Викторович, спасибо за вопрос. Да, дейс...",0.191153,0.585111,0.177821,0.056662,0.125933,0.011342,0.515630,0.160276,0.092698,0.073706,0.022987
1,duma 6,zero,У меня вопрос к Михаилу Васильевичу.Михаил Вас...,"Уважаемый Владимир Николаевич, вопрос о национ...",0.178169,0.500010,0.453272,0.042098,0.100889,0.027595,0.812877,0.187143,0.028446,0.055015,0.027595
2,duma 6,zero,"Вопрос представителю комитета, два вопроса. Пе...","Уважаемый Валентин Степанович, регионы, конечн...",0.270669,0.808077,0.168867,0.014514,0.112805,0.007131,0.607673,0.177821,0.100889,0.087574,0.012442
3,duma 6,zero,"Михаил Васильевич, в дополнение к уже сказанно...","Уважаемый Дмитрий Иванович, я думаю, что ваше ...",0.167226,0.607673,0.300756,0.044691,0.122533,0.013233,0.692652,0.377551,0.043376,0.090103,0.013233
4,duma 6,zero,"Спасибо.Уважаемый Александр Александрович, коп...",Что касается анализа комитета. Исходя из анали...,0.300982,0.731069,0.156115,0.039649,0.069552,0.022296,0.863402,0.191943,0.028446,0.051855,0.005230
5,duma 6,zero,"Вопрос Белякову.Уважаемый Сергей Юрьевич, в по...","Логики никакой не было, это действительно была...",0.304434,0.749097,0.307368,0.024433,0.075868,0.010024,0.538993,0.327678,0.109716,0.063725,0.014967
6,duma 6,zero,"Это не российское ноу-хау, весь мир идёт по пу...",...которая завозится на территорию магаданской...,0.209595,0.554480,0.275140,0.078088,0.115971,0.028446,0.414909,0.300756,0.233716,0.065615,0.011342
7,duma 6,zero,"Уважаемый Иван Иванович, уважаемые депутаты! О...","Спасибо.Уважаемый Олег Евгеньевич, я, в общем-...",0.246392,0.724880,0.239359,0.063725,0.065615,0.011342,0.569863,0.152042,0.103759,0.092698,0.023699
8,duma 6,zero,"Николай Васильевич, мы много размышляли, на са...","Сергей Владимирович, это вообще не власть, это...",0.398207,0.355785,0.164526,0.122533,0.250923,0.012442,0.777310,0.334599,0.050341,0.023699,0.008587
9,duma 6,zero,"Спасибо, Иван Иванович.У меня вопрос к Пантеле...","Виктор Евграфович, спасибо за вопрос. Пока ещё...",0.302761,0.760661,0.132974,0.109716,0.055015,0.007587,0.430157,0.245095,0.033096,0.132974,0.007826


In [83]:
#raw_data.to_csv('raw_data.csv', index=False)
#processed_data.to_csv('processed_data.csv', index=False)
#processed_data_nameless.to_csv('processed_data_nameless.csv', index=False)